[a,b] = [-1, 1];  q(x) = 1/eps, eps = 0.05, f(x) = (1/eps + pi^2)cos(pi*x), u = cos(pi*x) + exp((x-1)/eps^(1/2)) + exp(-(x+1)/eps^(1/2))

In [1]:
import numpy as np
np.set_printoptions(precision=2, suppress=True, linewidth=160)

In [2]:
from math import exp, cos, pi

a, b = -1, 1
eps = 0.05

q = 1/eps

f = lambda x: (1/eps + pi**2)*cos(pi*x) 

u = lambda x: cos(pi*x) + exp(((x-1)/eps**(1/2))) + exp(-(x+1)/eps**(1/2)) #точное решение


ua, ub = u(-1), u(1)

In [3]:
h = 0.1
N = int((b - a) / h)
nodes = np.arange(a,b+h,h)
fi = np.vectorize(f)(nodes)
qi = q
ui = np.vectorize(u)(nodes)
#print(nodes)


"""A = np.zeros((N+1, N+1))
A[0, 0] = 1
A[-1, -1] = 1
j = 0
for i in range(1, len(A)-1):
    A[i][j] = -1/(h**2)
    A[i][j+1] = 2/(h**2)+qi
    A[i][j+2] = -1/(h**2)
    j+=1
print(A)"""

#нижняя диагональ
A = np.zeros(N+1)
A[0], A[-1] = 0, 0
A[1:N] = -1/(h**2)
print('A:', A, len(A))

#главная диагональ
B = np.zeros(N+1)
B[0], B[-1] = 1, 1
B[1:N] = 2/(h**2)+qi
print('B:', B, len(B))

#верхняя диагональ
C = np.zeros(N+1)
C[0], C[-1] = 0, 0
C[1:N] = -1/(h**2)
print('C:', C, len(C))

#правая часть системы
F = np.zeros(N+1)
F[0], F[-1] = ua, ub
F[1], F[-2] = fi[1] + ua/(h**2), fi[-2] + ub/(h**2)
for i in range(2, len(F)-2):
    F[i] = fi[i]
print('F:', F, len(F))


A: [   0. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.    0.] 21
B: [  1. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220. 220.   1.] 21
C: [   0. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.    0.] 21
F: [  0.   -28.39 -24.17 -17.56  -9.23  -0.     9.23  17.56  24.17  28.41  29.87  28.41  24.17  17.56   9.23   0.    -9.23 -17.56 -24.17 -28.39   0.  ] 21


In [4]:
def progonka(A, B, C, F):
    """
    метод прогонки,
    A, B, C - диагональ под главной, главная дагональ и диагональ над главной соответственно,
    F - правая часть системы
    """
    n = len(F)
    alpha = np.zeros(n)
    beta = np.zeros(n)
    x = np.zeros(n)
    alpha[0] = -C[0]/B[0]
    beta[0] = F[0]/B[0]

    for i in range(1, n):
        alpha[i] = -C[i]/(A[i]*alpha[i-1] + B[i])
        beta[i] = (F[i] - A[i]*beta[i-1])/(A[i]*alpha[i-1] + B[i])

    x[n-1] = beta[n-1]

    for i in range(n-1, 0, -1):
        x[i-1] = alpha[i-1]*x[i] + beta[i-1]

    return x


In [5]:
ui

array([ 0.  , -0.31, -0.4 , -0.33, -0.14,  0.11,  0.38,  0.63,  0.84,  0.98,  1.02,  0.98,  0.84,  0.63,  0.38,  0.11, -0.14, -0.33, -0.4 , -0.31,  0.  ])

In [6]:
progonka(A, B, C, F)

array([ 0.  , -0.31, -0.4 , -0.32, -0.14,  0.11,  0.38,  0.64,  0.84,  0.98,  1.03,  0.98,  0.84,  0.64,  0.38,  0.11, -0.14, -0.32, -0.4 , -0.31,  0.  ])